In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

def simulate_transition(horizontal_thrust_max=30, vertical_thrust_start=100, max_pitch_deg=18, drag_coeff=0.3):
    g = 9.81
    rho = 1.225

    mass = 10.0
    wing_area = 1.2
    CL = 0.8
    CD = drag_coeff
    stall_speed = 8.0
    eta = 0.7 # Efficiency of the propulsion system

    dt = 0.1
    T_end = 50.0
    time = np.arange(0, T_end, dt)

    vx = np.zeros_like(time)
    vz = np.zeros_like(time)
    alt = np.zeros_like(time)
    pitch = np.clip(np.linspace(0, np.deg2rad(max_pitch_deg), len(time)), 0, np.deg2rad(max_pitch_deg))
    x = np.zeros_like(time)

    horizontal_thrust = np.clip(np.linspace(0, horizontal_thrust_max, len(time)), 0, horizontal_thrust_max)
    vertical_thrust = np.clip(np.linspace(vertical_thrust_start, 0, len(time)), 0, vertical_thrust_start)
    
    energy_vertical = 0.0
    energy_horizontal = 0.0
    
    for i in range(1, len(time)):
        speed = np.sqrt(vx[i-1]**2 + vz[i-1]**2)
        lift = 0.5 * rho * vx[i-1]**2 * wing_area * CL
        drag = 0.5 * rho * vx[i-1]**2 * wing_area * CD

        Tv = vertical_thrust[i]
        Th = horizontal_thrust[i]

        Fz = Tv - mass * g + lift * np.cos(pitch[i])
        Fy = Th * np.sin(pitch[i])
        az = (Fz + Fy) / mass
        vz[i] = vz[i-1] + az * dt
        alt[i] = alt[i-1] + vz[i] * dt

        ax = (Th * np.cos(pitch[i]) + lift * np.sin(pitch[i]) - drag) / mass
        vx[i] = vx[i-1] + ax * dt
        x[i] = x[i-1] + vx[i] * dt
        
     # Energy Estimation
        # Estimate induced velocity for vertical thrust
        rotor_area = 0.2  # m² (adjust based on your prop setup)
        if Tv > 0:
            v_induced = np.sqrt(Tv / (2 * rho * rotor_area))
            # print(f"Induced Velocity: {v_induced:.2f} m/s")
            P_vert = (Tv * v_induced) / eta
        else:
            # print("No vertical thrust applied, no induced velocity.")
            P_vert = 0
        v_eff_horizontal = abs(vx[i])
        P_horiz = (Th * v_eff_horizontal) / eta
        energy_vertical += P_vert * dt
        energy_horizontal += P_horiz * dt

    plt.figure(figsize=(12, 8))

    plt.subplot(3, 1, 1)
    plt.plot(time, alt, label="Altitude (m)")
    plt.ylabel("Altitude (m)")
    plt.grid()
    plt.legend()

    plt.subplot(3, 1, 2)
    plt.plot(time, vx, label="Horizontal Velocity (m/s)")
    plt.axhline(stall_speed, color='r', linestyle='--', label="Stall Speed")
    plt.ylabel("Vx (m/s)")
    plt.grid()
    plt.legend()

    plt.subplot(3, 1, 3)
    plt.plot(time, np.rad2deg(pitch), label="Pitch (deg)")
    plt.ylabel("Pitch Angle (deg)")
    plt.xlabel("Time (s)")
    plt.grid()
    plt.legend()

    plt.tight_layout()
    plt.show()
    E_total = energy_vertical + energy_horizontal
    print(f"🔋 Estimated Energy Consumption:")
    print(f" - Vertical System: {energy_vertical / 1000:.4f} kJ")
    print(f" - Horizontal System: {energy_horizontal / 1000:.4f} kJ")
    print(f" - Total Energy: {E_total / 1000:.2f} kJ")
interact(simulate_transition,
         horizontal_thrust_max=FloatSlider(value=20, min=10, max=50, step=1, description='Max H-Thrust (N)'),
         vertical_thrust_start=FloatSlider(value=81200, min=80, max=160, step=1, description='Start V-Thrust (N)'),
         max_pitch_deg=FloatSlider(value=20, min=5, max=45, step=1, description='Max Pitch (deg)'),
         drag_coeff=FloatSlider(value=0.3, min=0.05, max=1.0, step=0.05, description='Drag Coeff (CD)'))


interactive(children=(FloatSlider(value=20.0, description='Max H-Thrust (N)', max=50.0, min=10.0, step=1.0), F…

<function __main__.simulate_transition(horizontal_thrust_max=30, vertical_thrust_start=100, max_pitch_deg=18, drag_coeff=0.3)>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

def simulate_transition(horizontal_thrust_max=30, vertical_thrust_start=100, max_pitch_deg=18, T_end=20):
    g = 9.81
    rho = 1.225

    mass = 10.0
    wing_area = 1.5
    # Aerodynamic coefficients (airfoil-based)
    CL0 = 0.2           # Base lift coefficient at 0 AoA
    CL_alpha = 5.7      # Lift curve slope (per radian)
    CD0 = 0.02          # Zero-lift drag coefficient
    k = 0.04            # Induced drag factor
    stall_speed = 8.0

    eta = 0.7 # Efficiency of the propulsion system
    
    z_target = 50  # meters (target cruise altitude)
    Kp_z = 1.5     # proportional gain for altitude control

    dt = 0.1
    T_end = T_end  # seconds
    time = np.arange(0, T_end, dt)

    vx = np.zeros_like(time)
    vz = np.zeros_like(time)
    alt = np.zeros_like(time)
    pitch = np.clip(np.linspace(0, np.deg2rad(max_pitch_deg), len(time)), 0, np.deg2rad(max_pitch_deg))
    x = np.zeros_like(time)
    ngsx = np.zeros_like(time)  # g's experienced during maneuver
    ngsy = np.zeros_like(time)  # g's experienced during maneuver

    horizontal_thrust = np.clip(np.linspace(0, horizontal_thrust_max, len(time)), 0, horizontal_thrust_max)
    vertical_thrust = np.clip(np.linspace(vertical_thrust_start, 0, len(time)), 0, vertical_thrust_start)
    
    energy_vertical = 0.0
    energy_horizontal = 0.0
    
    for i in range(1, len(time)):
        # Compute flight path angle
        flight_path_angle = np.arctan2(vz[i-1], vx[i-1] + 1e-4)  # Avoid division by zero
        alpha = pitch[i] - flight_path_angle  # AoA

        # Limit AoA to realistic range (avoid extreme stall in this simple model)
        alpha = np.clip(alpha, np.deg2rad(-5), np.deg2rad(15))

        # Update CL and CD based on AoA
        CL = CL0 + CL_alpha * alpha
        CD = CD0 + k * CL**2
        # print(f"{vx[i-1], vz[i-1], alt[i-1], pitch[i], flight_path_angle, alpha, CL, CD}")
        speed = np.sqrt(vx[i-1]**2 + vz[i-1]**2)
        lift = 0.5 * rho * speed**2 * wing_area * CL
        drag = 0.5 * rho * speed**2 * wing_area * CD
        

        Tv = vertical_thrust[i]
        Th = horizontal_thrust[i]
        
        # ngsx[i] = (-1*lift * np.sin(alpha) - drag * np.cos(alpha) + Th) / (mass * g * np.sin(pitch[i]))  # g's experienced
        ngsy[i-1] = (lift * np.cos(alpha) - drag * np.sin(alpha) + Tv ) / (mass * g * np.cos(pitch[i]))  # g's experienced
        if i == 1:
            print(mass * g * np.cos(pitch[i]), lift * np.cos(alpha), drag * np.sin(alpha), Tv)
            print(f"Initial G's: {ngsy[i]:.2f} g's")
        
        Fx = -Tv * np.sin(pitch[i]) + Th * np.cos(pitch[i]) - drag * np.cos(flight_path_angle) + -1*lift * np.sin(flight_path_angle)
        Fz = Th * np.sin(pitch[i]) + Tv * np.cos(pitch[i]) - mass * g + lift * np.cos(flight_path_angle) - drag * np.sin(flight_path_angle)
        az = Fz / mass
        
        
        vz[i] = max(vz[i-1] + az * dt, 0)
        alt[i] = max(alt[i-1] + vz[i] * dt, 0)

        ax = Fx / mass
        vx[i] = vx[i-1] + ax * dt
        x[i] = x[i-1] + vx[i] * dt
        
        max_speed = 18  # m/s, adjust based on your flight model
        vx[i] = np.clip(vx[i], -max_speed, max_speed)
        vz[i] = np.clip(vz[i], -max_speed, max_speed)
        
     # Energy Estimation
        # Estimate induced velocity for vertical thrust
        rotor_area = 0.2  # m² (adjust based on your prop setup)
        if Tv > 0:
            v_induced = np.sqrt(Tv / (2 * rho * rotor_area))
            # print(f"Induced Velocity: {v_induced:.2f} m/s")
            P_vert = (Tv * v_induced) / eta
        else:
            # print("No vertical thrust applied, no induced velocity.")
            P_vert = 0
        v_eff_horizontal = abs(vx[i])
        P_horiz = (Th * v_eff_horizontal) / eta
        energy_vertical += P_vert * dt
        energy_horizontal += P_horiz * dt
    E_total = energy_vertical + energy_horizontal
    P_hover = vertical_thrust_start * np.sqrt(vertical_thrust_start / (2 * rho * rotor_area)) / eta
    print(f"Hover Power: {P_hover:.2f} W")
    print(f"Energy consumption hover: {P_hover * 20 / 1000:.2f} kJ")
    
    
    print(f"🔋 Estimated Energy Consumption:")
    print(f" - Vertical System: {energy_vertical / 1000:.4f} kJ")
    print(f" - Horizontal System: {energy_horizontal / 1000:.4f} kJ")
    print(f" - Total Energy: {E_total / 1000:.2f} kJ")
    
    plt.figure(figsize=(12, 8))

    plt.subplot(5, 1, 1)
    plt.plot(time, alt, label="Altitude (m)")
    plt.ylabel("Altitude (m)")
    plt.grid()
    plt.legend()

    plt.subplot(5, 1, 2)
    plt.plot(time, vx, label="Horizontal Velocity (m/s)")
    plt.plot(time, vz, label="Vertical Velocity (m/s)")
    plt.axhline(stall_speed, color='r', linestyle='--', label="Stall Speed")
    plt.ylabel("Vx (m/s)")
    plt.grid()
    plt.legend()
    
    

    plt.subplot(5, 1, 3)
    plt.plot(time, np.rad2deg(pitch), label="Pitch (deg)")
    plt.ylabel("Pitch Angle (deg)")
    plt.xlabel("Time (s)")
    plt.grid()
    plt.legend()
    
    plt.subplot(5, 1, 4)
    plt.plot(time, np.rad2deg(pitch), label='Pitch (deg)')
    plt.plot(time, np.rad2deg(np.array(pitch) - np.arctan2(vz, vx + 1e-6)), label='AoA (deg)')
    plt.title("Pitch and Angle of Attack During Transition")
    plt.xlabel("Time (s)")
    plt.ylabel("Angle (degrees)")
    plt.legend()
    plt.grid(True)
    
    plt.subplot(5, 1, 5)
    plt.plot(time, ngsy, label="G's Experienced")
    plt.ylabel("G's")
    plt.xlabel("Time (s)")
    plt.grid()
    plt.legend()

    plt.tight_layout()
    plt.show()
interact(simulate_transition,
         horizontal_thrust_max=FloatSlider(value=50, min=10, max=100, step=1, description='Max H-Thrust (N)'),
         vertical_thrust_start=FloatSlider(value=120, min=80, max=160, step=1, description='Start V-Thrust (N)'),
         max_pitch_deg=FloatSlider(value=10, min=-5, max=30, step=1, description='Max Pitch (deg)'),
         T_end=FloatSlider(value=20, min=5, max=50, step=0.5, description='Time End (s)'))

interactive(children=(FloatSlider(value=50.0, description='Max H-Thrust (N)', min=10.0, step=1.0), FloatSlider…

<function __main__.simulate_transition(horizontal_thrust_max=30, vertical_thrust_start=100, max_pitch_deg=18, T_end=20)>

: 